<a href="https://colab.research.google.com/github/felixpetschko/scirpy/blob/main/quantum_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Short project description by Felix

**Biological use case:**

We have a cancer peptide and we want to find possible CDR3 T cell receptor sequences that would bind to it. If we can find such a sequence, it could be used in TCR-engineered T cell therapy.

**Approach:**

We have amino acid sequence (20 possible amino acids), but we divide them into 4 different classes based on physical properties:
* hydrophobic (H)
* polar (P)
* positively charged (+)
* negatively charged (-)

We can define certain boolean rules that should be true for a CDR3 sequence that would bind to a specific peptide. E.g, it should start and end with a hydrophobic amino acid and there should be at least 2 polar ones and so on. Then we want to search for possible candidate solution with grover's search algorithm.

These rules should be used for our oracle that decides if a CDR3 sequence is a "correct" solution or not. If the rules can be written down in boolean logic it's easier to use them for the oracle than most other approaches.

It turns out that we can have sequences of physical properties of length 5 with this approach, e.g. ['P', '+', 'P', 'H', '-']

As candidate solutions, we could get a result like this.

['H', '-', 'P', 'P', 'H']

['H', '+', 'P', 'H', 'H']

['H', '+', 'P', 'P', 'H']

['H', '-', 'P', 'H', 'H']

This can be decoded back into amino acids. There exists several possible amino acid decoding for one string of physical properties.

With earlier project ideas we had the problem that already preparing the grover's search algorithm with a certain list of items takes O(n). If we search in the a space of all possible CDR3 sequences of a certain length however, we don't have this preparation problem.

The current approach could be easily extended to stronger hardware by using longer sequence and more complex rules.

## Longer project description (generated by ChatGPT)

### Motivation and Biological Context
T cell receptors (TCRs) play a central role in adaptive immunity by recognizing short peptide fragments presented on major histocompatibility complex (MHC) molecules. In cancer immunotherapy, a key goal is to identify or design TCRs that specifically recognize tumor-associated peptides while avoiding off-target binding to healthy tissue. Successful identification of such TCRs can enable downstream therapeutic strategies such as TCR-engineered T cell therapy, where patient T cells are genetically modified to express a tumor-reactive receptor and then reinfused.

The most variable and specificity-determining part of the TCR is the CDR3 (Complementarity-Determining Region 3) loop. Its amino-acid composition and physicochemical properties largely determine peptide binding. However, the combinatorial diversity of possible CDR3 sequences is enormous, making exhaustive enumeration or brute-force design infeasible even classically, let alone biologically.

This project explores whether quantum search techniques, specifically Grover’s algorithm, can be used as a conceptual framework to search for candidate CDR3 patterns that satisfy biologically motivated constraints relative to a given cancer peptide.

---

### Abstracting the Biological Problem
Rather than working directly at the amino-acid level, we introduce a coarse-grained encoding of CDR3 sequences. Each CDR3 position is represented not by a specific amino acid, but by one of four physicochemical classes:

- `H` — hydrophobic  
- `P` — polar  
- `+` — positively charged  
- `−` — negatively charged  

Each class is encoded using 2 qubits, allowing a compact quantum representation. For example, a 5-position CDR3 core is represented using 10 data qubits. This abstraction captures biologically relevant properties (hydrophobic anchoring, electrostatic complementarity, flexibility) while keeping the search space small enough to fit on near-term quantum hardware.

Importantly, the project does not assume a predefined list of candidate sequences. Instead, it defines a rule-based search space: a CDR3 candidate is considered valid if it satisfies a set of interpretable biological rules derived from immunological heuristics.

---

### Rule-Based Search Space
The rules encode simplified but biologically motivated constraints, such as:

- hydrophobic residues at the ends of the CDR3 loop (anchoring and structural stability)
- a polar residue in the center (flexibility and specificity)
- electrostatic complementarity between the cancer peptide core and the CDR3 contact region
- avoidance of overly charged motifs that could lead to nonspecific binding

Crucially, these rules define a **predicate**, not a list. We do not enumerate valid CDR3 sequences in advance; instead, a candidate is accepted or rejected algorithmically. This distinction matters because Grover’s algorithm assumes access to an oracle that can verify a candidate, not construct solutions directly.

---

### Oracle Formulation
To apply Grover’s algorithm, the biological rules must be translated into a Boolean predicate

$$
f(x) \in \{0,1\},
$$

where $x$ is a binary encoding of a CDR3 pattern. The oracle marks exactly those quantum basis states corresponding to candidates that satisfy all rules.

In this project, the oracle is implemented as a phase oracle:

$$
|x\rangle \rightarrow (-1)^{f(x)} |x\rangle.
$$

**Key design constraints**
- the oracle must be diagonal in the computational basis
- the predicate must be implemented reversibly
- no classical precomputation or enumeration of all valid solutions is allowed
- the total qubit count is restricted to 12 qubits, reflecting realistic near-term hardware limits

To enable nontrivial logical combinations of rules (AND conditions, parity constraints, peptide-dependent logic), the circuit uses:
- **10 data qubits** (CDR3 encoding)
- **1 work ancilla** (clean $|0\rangle$, used for intermediate computation)
- **1 phase ancilla** prepared in $|-\rangle$ for phase kickback

This structure allows the oracle to compute biologically meaningful predicates while remaining a valid Grover oracle.

---

### Quantum Search and Interpretation
Grover’s algorithm is then used to amplify the amplitudes of all CDR3 patterns that satisfy the biological rules. Depending on how restrictive the rules are, the algorithm can be tuned to amplify:

- a single candidate (design-like setting), or
- a small set of candidates (screening-like setting)

The output of the quantum circuit is not a final therapeutic sequence, but a set of coarse-grained CDR3 patterns enriched for biologically plausible peptide binders. These patterns could then be refined classically into concrete amino-acid sequences and evaluated using structural modeling, machine learning predictors, or experimental assays.

---

### Resulting Therapeutic Use Case
In a realistic pipeline, this approach would serve as an early-stage candidate generation or filtering step:

1. Start with a known cancer-associated peptide.
2. Use a quantum-assisted rule-based search to identify plausible CDR3 patterns.
3. Expand these patterns into specific amino-acid sequences.
4. Evaluate binding and specificity using classical computational methods.
5. Select top candidates for experimental validation and TCR engineering.

While current quantum hardware limits this approach to small, abstracted problems, the project demonstrates how biological reasoning can be translated into quantum oracles, and how quantum search can be meaningfully integrated into immunotherapy-related design tasks.

---

### Conceptual Takeaway
This project does not claim a quantum advantage in practice today. Instead, it demonstrates:

- how biological constraints can be encoded as Boolean predicates
- why Grover’s algorithm naturally fits verification-based biological searches
- why rule-based formulations are essential when explicit enumeration is infeasible
- how quantum computing concepts can be aligned with realistic biomedical questions

In that sense, the work serves both as a technical exploration of oracle design and as a conceptual bridge between quantum algorithms and computational immunology.


## Rule summary (candidate validity)

### Candidate encoding
A candidate is a **10-bit string** on qubits `q0..q9` representing **5 positions** `pos0..pos4`.  
Each position uses **2 bits** \((a_i, b_i)\):

- `pos0`: \((a_0=q0,\; b_0=q1)\)
- `pos1`: \((a_1=q2,\; b_1=q3)\)
- `pos2`: \((a_2=q4,\; b_2=q5)\)
- `pos3`: \((a_3=q6,\; b_3=q7)\)
- `pos4`: \((a_4=q8,\; b_4=q9)\)

**2-bit class code (per position)**
- `00` = `H` (hydrophobic)
- `01` = `P` (polar)
- `10` = `+` (positively charged)
- `11` = `-` (negatively charged)

---

### Peptide-dependent preprocessing
Before checking a candidate, compute:

1. `core` = central 3 residues of the peptide (e.g. `"GLCTLVAML"` \(\rightarrow\) `"TLV"`).
2. `sign` = `peptide_charge_sign(core)` based on counts in `core`:
   - positive residues: `K`, `R`
   - negative residues: `D`, `E`

Returns:
- `"neg"` if \(\#DE > \#KR\)
- `"pos"` if \(\#KR > \#DE\)
- `"none"` otherwise

This `sign` only affects **Rule R4** below.

---

## Rules (R1–R5)
A candidate is **valid iff all rules hold**.

### R1: `pos0` must be hydrophobic
- Class form: `pos0 = H`
- Bit form: \((a_0,b_0)=(0,0)\)  i.e. `q0=0` and `q1=0`

### R2: `pos4` must be hydrophobic
- Class form: `pos4 = H`
- Bit form: \((a_4,b_4)=(0,0)\)  i.e. `q8=0` and `q9=0`

### R3: `pos2` must be polar
- Class form: `pos2 = P`
- Bit form: \((a_2,b_2)=(0,1)\)  i.e. `q4=0` and `q5=1`

### R5: `pos3` must be **not charged**
Implemented as: \(a_3 = 0\).

- Bit form: `q6 = 0`
- Class form: `pos3 ∈ {H, P}`  
  because `H=00` and `P=01` have \(a_3=0\), while `+=10` and `-=11` have \(a_3=1\) (forbidden).  
  (`q7` is free, selecting between `H` and `P` when `q6=0`.)

### R4: `pos1` must satisfy a **peptide-conditioned charge rule**
This depends on `sign` from the peptide core:

- If `sign == "neg"`: enforce `pos1 = +`
  - bits: \((a_1,b_1)=(1,0)\)  i.e. `q2=1`, `q3=0`

- If `sign == "pos"`: enforce `pos1 = -`
  - bits: \((a_1,b_1)=(1,1)\)  i.e. `q2=1`, `q3=1`

- If `sign == "none"`: enforce `pos1` is **charged** (`+` or `-`)
  - bits: \(a_1=1\)  i.e. `q2=1` (and `q3` free)

---

## One-line validity condition
Valid iff:

- `pos0 = H`
- `pos4 = H`
- `pos2 = P`
- `pos3 ∈ {H, P}` (not charged)
- `pos1` is charge-complementary to peptide core sign:
  - `"neg" → +`
  - `"pos" → -`
  - `"none" → {+ or -}`

---

## Example: peptide `"GLCTLVAML"`
Core is `"TLV"`, which has no `K/R/D/E`, so `sign = "none"`.

Therefore valid patterns are:

- `pos0 = H`
- `pos1 ∈ {+, -}`
- `pos2 = P`
- `pos3 ∈ {H, P}`
- `pos4 = H`

Number of solutions:

$$
M = 1 \cdot 2 \cdot 1 \cdot 2 \cdot 1 = 4.
$$


## Function descriptions

### `peptide_core(peptide: str, k: int = 3) -> str`
Extracts the peptide “core” (default length $k = 3$) by taking the central substring.

**Input**
- `peptide`: peptide sequence string (e.g. `"GLCTLVAML"`)

**Output**
- central $k$ residues (e.g. `"TLV"`)

**Purpose**  
Uses a simple peptide-derived feature to condition the CDR3 rules (toy version of peptide-aware design).

---

### `peptide_charge_sign(core: str) -> str`
Computes a coarse charge dominance label for the peptide core.

**Counts**
- positive residues: `K`, `R`
- negative residues: `D`, `E`

**Returns**
- `"neg"` if number of `D`/`E` is greater than number of `K`/`R`
- `"pos"` if number of `K`/`R` is greater than number of `D`/`E`
- `"none"` otherwise

**Purpose**  
Chooses which electrostatic rule to enforce on the CDR3 pattern.

---

### `decode_10bit_to_classes(data10_q0_to_q9: str) -> List[str]`
Decodes a 10-bit data string into a 5-position class pattern.

**Mapping**  
Each pair of bits is interpreted as one class:
- `00` -> `H` (hydrophobic)
- `01` -> `P` (polar)
- `10` -> `+` (positively charged)
- `11` -> `-` (negatively charged)

**Example**
- Input: `"0010010100"`
- Pairs: `00 10 01 01 00`
- Output: `["H", "+", "P", "P", "H"]`

**Purpose**  
Makes quantum measurement results human-readable.

---

### `predicate_classical(data10: str, peptide: str) -> bool`
Implements the rule-based oracle as a classical Boolean function.

**Input**
- `data10`: 10-bit candidate encoding (qubits `q0..q9`)
- `peptide`: peptide sequence

**Output**
- `True` if the candidate satisfies all rules  
- `False` otherwise

**Purpose**  
This function defines the ground-truth notion of validity.  
The quantum oracle must implement exactly the same logic, otherwise Grover’s algorithm will not amplify the intended solution set.

---

### `brute_force_solutions(peptide: str, limit_print: int = 12) -> (int, List[str])`
Iterates over all possible candidates: $2^{10} = 1024$.  
Selects those that satisfy `predicate_classical`.

**Returns**
- `M`: number of valid solutions
- `sols`: list of all valid 10-bit strings

Also prints a few decoded examples.

**Purpose**  
Used for debugging and calibration:
- checks whether the biological rules are too loose or too strict
- helps choose a sensible number of Grover iterations
- provides a reference solution set to compare against quantum output

---

### `phase_oracle_predicate(peptide: str) -> QuantumCircuit`
Builds the quantum phase oracle that flips the phase of basis states encoding valid candidates.

**Concept**  
Conceptually implements the transformation:

$$
|x\rangle \rightarrow (-1)^{f(x)} |x\rangle
$$

where $f(x)$ is the same predicate as `predicate_classical`.

**Mechanics**
- rule conditions are enforced using reversible logic
- a phase ancilla prepared in the $|-\rangle$ state is used
- a controlled-`X` acting on $|-\rangle$ produces a phase flip (phase kickback)
- temporary `X`-gates used to implement zero-controls are uncomputed

**Purpose**  
This is the key quantum component of the project.  
If this oracle does not exactly match the classical predicate, Grover’s algorithm will not work correctly.

---

### `diffuser_on_data_10() -> QuantumCircuit`
Creates the Grover diffuser (inversion about the mean) acting only on the 10 data qubits.

**Concept**  
Conceptually performs:

$$
D = 2|s\rangle\langle s| - I
$$

where $|s\rangle$ is the uniform superposition over all candidates.

**Purpose**  
This is the second half of each Grover iteration.  
It amplifies the amplitudes of states marked by the oracle.

---

### `build_grover_circuit(peptide: str, iters: int) -> QuantumCircuit`
Assembles the full Grover search circuit.

**Steps**
1. Prepare the data register in uniform superposition using Hadamard gates
2. Prepare the phase ancilla in the $|-\rangle$ state
3. Repeat `iters` times:
   1. apply the oracle
   2. apply the diffuser
4. Measure the 10 data qubits

**Purpose**  
This is the circuit that is executed on a simulator or quantum hardware.

---

## How to interpret the results

### Meaning of a “solution”
A measured bitstring corresponds to a CDR3 class-pattern, not to a concrete amino-acid sequence.

**Example**
- `00 10 01 01 00` $\rightarrow$ `H, +, P, P, H`

**Interpretation**
- positions 0 and 4 are hydrophobic anchors
- position 1 is positively charged
- the center is polar and flexible
- additional rules may impose further constraints

To obtain a concrete CDR3 sequence, each class would later be mapped to a specific amino acid  
(e.g. `H` -> `L`, `P` -> `S`, `+` -> `K`, `-` -> `E`) and embedded into a longer CDR3 template.

---

### Baseline probability versus Grover amplification
Let:
- $N = 1024$ (all possible candidates)
- $M =$ number of valid candidates from brute-force enumeration

Uniform random sampling would produce valid candidates with probability $M/N$.  
After Grover’s algorithm, the fraction of measured valid candidates should be significantly larger than $M/N$.

If the fraction is unchanged, the oracle or iteration count is likely incorrect.

---

### Multiple solutions are expected
If $M > 1$, Grover amplifies the entire set of valid candidates, not a single bitstring.

This means:
- several different valid bitstrings appear
- no single result necessarily dominates
- success is measured by enrichment of valid candidates, not uniqueness

---

### Choosing the number of Grover iterations
A useful heuristic is:

$$
k \approx \frac{\pi}{4}\sqrt{\frac{N}{M}}
$$

If $M$ is large, only a few iterations (1 to 3) are needed.  
If $M$ is small, more iterations are required.

---

### Bitstring ordering
Qiskit often reports measured bitstrings in reversed order.  
Always ensure consistent decoding (e.g. reversing the string before interpretation).

---

### Simulator versus real hardware
On a simulator:
- clear enrichment of valid candidates should be visible

On real quantum hardware:
- deep multi-controlled gates are noisy
- enrichment may be weaker or absent
- qualitative agreement with the simulator at small depth is considered a success


In [23]:
!pip install qiskit

In [5]:
pip install qiskit-aer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 123.6 MB/s eta 0:00:00


In [24]:
import math
from typing import List, Tuple

from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import MCXGate
from qiskit_aer import Aer

# -------------------------
# 2-bit class encoding per position
# -------------------------
# 00 -> H (hydrophobic)
# 01 -> P (polar)
# 10 -> + (positively charged)
# 11 -> - (negatively charged)
BITS_TO_CLASS = {"00": "H", "01": "P", "10": "+", "11": "-"}
CLASS_TO_BITS = {v: k for k, v in BITS_TO_CLASS.items()}

# Data qubits (10): 5 positions × 2 bits
# pos0: q0=a0, q1=b0
# pos1: q2=a1, q3=b1
# pos2: q4=a2, q5=b2
# pos3: q6=a3, q7=b3
# pos4: q8=a4, q9=b4
#
# Ancillas:
# q10 = work (unused in this version, but kept for your 12-qubit budget)
# q11 = phase ancilla (prepared in |->)

CH_POS = set("KR")
CH_NEG = set("DE")

def peptide_core(peptide: str, k: int = 3) -> str:
    p = peptide.strip().upper()
    if len(p) <= k:
        return p
    start = (len(p) - k) // 2
    return p[start:start + k]

def peptide_charge_sign(core: str) -> str:
    pos = sum(aa in CH_POS for aa in core)
    neg = sum(aa in CH_NEG for aa in core)
    if neg > pos:
        return "neg"
    if pos > neg:
        return "pos"
    return "none"

def decode_10bit_to_classes(data10_q0_to_q9: str) -> List[str]:
    return [BITS_TO_CLASS[data10_q0_to_q9[i:i+2]] for i in range(0, 10, 2)]

# -------------------------
# New "more biological" predicate (classical)
# -------------------------
def predicate_classical(data10: str, peptide: str) -> bool:
    """
    More biologically interpretable toy rules:

    R1: pos0 == H
    R2: pos4 == H
    R3: pos2 == P
    R4: peptide-conditioned electrostatic contact:
        - if peptide core net neg -> pos1 == +
        - if peptide core net pos -> pos1 == -
        - if neutral -> pos1 is charged (+ or -)
    R5: pos3 not charged (avoid overly charged core): a3 == 0
    """
    if len(data10) != 10 or any(ch not in "01" for ch in data10):
        return False

    # unpack bits
    a0, b0 = int(data10[0]), int(data10[1])
    a1, b1 = int(data10[2]), int(data10[3])
    a2, b2 = int(data10[4]), int(data10[5])
    a3, b3 = int(data10[6]), int(data10[7])
    a4, b4 = int(data10[8]), int(data10[9])

    # R1/R2: anchors
    pos0_is_H = (a0 == 0 and b0 == 0)
    pos4_is_H = (a4 == 0 and b4 == 0)

    # R3: center polar (01)
    pos2_is_P = (a2 == 0 and b2 == 1)

    # R5: pos3 not charged -> a3 == 0
    pos3_not_charged = (a3 == 0)

    # R4: peptide-conditioned pos1 charge/sign
    core = peptide_core(peptide, 3)
    sign = peptide_charge_sign(core)

    if sign == "neg":
        # pos1 == +  (10)
        pos1_ok = (a1 == 1 and b1 == 0)
    elif sign == "pos":
        # pos1 == -  (11)
        pos1_ok = (a1 == 1 and b1 == 1)
    else:
        # neutral -> pos1 charged (a1 == 1), b1 can be 0/1
        pos1_ok = (a1 == 1)

    return pos0_is_H and pos4_is_H and pos2_is_P and pos1_ok and pos3_not_charged


def brute_force_solutions(peptide: str, limit_print: int = 12) -> Tuple[int, List[str]]:
    sols = []
    for x in range(2**10):
        s = format(x, "010b")  # q0..q9 order
        if predicate_classical(s, peptide):
            sols.append(s)

    print(f"[bruteforce] Peptide={peptide} core={peptide_core(peptide,3)} sign={peptide_charge_sign(peptide_core(peptide,3))}")
    print(f"[bruteforce] M = {len(sols)} solutions out of N = 1024")
    for s in sols[:limit_print]:
        print(" ", s, "->", decode_10bit_to_classes(s))
    if len(sols) > limit_print:
        print(f"... (showing first {limit_print} of {len(sols)})")
    return len(sols), sols

# -------------------------
# Quantum phase oracle for the SAME predicate (no enumeration)
# -------------------------
def phase_oracle_predicate(peptide: str) -> QuantumCircuit:
    """
    Implements a single phase flip (via phase ancilla |->) iff predicate holds.

    Uses only data qubits as controls (work ancilla q10 left unused).
    """
    qc = QuantumCircuit(12, name="Oracle(bio)")

    # Indices
    a0, b0 = 0, 1
    a1, b1 = 2, 3
    a2, b2 = 4, 5
    a3, b3 = 6, 7
    a4, b4 = 8, 9
    phase = 11

    # Determine which pos1 condition applies
    core = peptide_core(peptide, 3)
    sign = peptide_charge_sign(core)

    # We'll build a list of control qubits that must be |1>.
    # For bits we want to be 0, we apply X first (0-control -> 1-control).

    # Desired literals:
    # pos0 == H => a0=0, b0=0  => X(a0), X(b0), then control on a0,b0
    # pos4 == H => a4=0, b4=0  => X(a4), X(b4), then control on a4,b4
    # pos2 == P => a2=0, b2=1  => X(a2) then control on a2 and b2
    # pos3 not charged => a3=0  => X(a3) then control on a3
    #
    # pos1 depends:
    #   neg -> a1=1,b1=0  => control on a1 and X(b1) then control b1
    #   pos -> a1=1,b1=1  => control on a1,b1
    #   none -> a1=1      => control on a1 only

    # Apply X for 0-controls
    qc.x(a0); qc.x(b0)
    qc.x(a4); qc.x(b4)
    qc.x(a2)           # a2=0
    qc.x(a3)           # a3=0

    controls = [a0, b0, a4, b4, a2, b2, a3]  # b2 must be 1 so no X there

    # pos1 controls
    if sign == "neg":
        # b1 must be 0
        qc.x(b1)
        controls += [a1, b1]
    elif sign == "pos":
        controls += [a1, b1]
    else:
        controls += [a1]

    # Flip phase using phase ancilla |-> :
    # controlled-X on |-> is a phase flip of the controls-subspace.
    qc.append(MCXGate(len(controls)), controls + [phase])

    # Undo any X flips (uncompute literal mappings)
    if sign == "neg":
        qc.x(b1)

    qc.x(a3)
    qc.x(a2)
    qc.x(b4); qc.x(a4)
    qc.x(b0); qc.x(a0)

    return qc

# -------------------------
# Grover diffuser on data register only (q0..q9)
# -------------------------
def diffuser_on_data_10() -> QuantumCircuit:
    qc = QuantumCircuit(12, name="Diffuser(data10)")
    data = list(range(10))

    qc.h(data)
    qc.x(data)

    target = data[-1]
    controls = data[:-1]
    qc.h(target)
    qc.append(MCXGate(len(controls)), controls + [target])
    qc.h(target)

    qc.x(data)
    qc.h(data)
    return qc

# -------------------------
# Build Grover circuit
# -------------------------
def build_grover_circuit(peptide: str, iters: int) -> QuantumCircuit:
    oracle = phase_oracle_predicate(peptide)
    diff = diffuser_on_data_10()

    qc = QuantumCircuit(12, 10)  # measure only q0..q9
    data = list(range(10))
    phase = 11

    # data superposition
    qc.h(data)

    # prepare phase ancilla in |-> (must be done ONCE outside the oracle)
    qc.x(phase)
    qc.h(phase)

    for _ in range(iters):
        qc.append(oracle, range(12))
        qc.append(diff, range(12))

    qc.measure(data, list(range(10)))
    assert qc.num_qubits == 12
    return qc

# -------------------------
# Run (simulator) + compare to brute force
# -------------------------
if __name__ == "__main__":
    peptide = "GLCTLVAML"  # replace

    M, sols = brute_force_solutions(peptide, limit_print=8)
    N = 2**10

    # Good iteration heuristic
    iters = max(1, int(round((math.pi / 4) * math.sqrt(N / max(1, M)))))
    print(f"[grover] Using iters = {iters} (based on M={M})")

    qc = build_grover_circuit(peptide, iters=iters)
    print("[grover] Circuit qubits:", qc.num_qubits)

    backend = Aer.get_backend("aer_simulator")
    tqc = transpile(qc, backend=backend, optimization_level=1)
    res = backend.run(tqc, shots=2048).result()
    counts = res.get_counts()

    def key_to_q0_to_q9(key: str) -> str:
        return key[::-1]

    top = sorted(counts.items(), key=lambda kv: kv[1], reverse=True)[:15]
    print("\nTop measured results:")
    for key, ct in top:
        data10 = key_to_q0_to_q9(key)
        ok = predicate_classical(data10, peptide)
        tag = "VALID" if ok else ""
        print(f"{key}  {ct:4d}  q0..q9={data10}  {decode_10bit_to_classes(data10)}  {tag}")

    valid_shots = 0
    for key, ct in counts.items():
        data10 = key_to_q0_to_q9(key)
        if predicate_classical(data10, peptide):
            valid_shots += ct
    print(f"\nVALID shots = {valid_shots} / 2048  ({valid_shots/2048:.3f})")


[bruteforce] Peptide=GLCTLVAML core=TLV sign=none
[bruteforce] M = 4 solutions out of N = 1024
  0010010000 -> ['H', '+', 'P', 'H', 'H']
  0010010100 -> ['H', '+', 'P', 'P', 'H']
  0011010000 -> ['H', '-', 'P', 'H', 'H']
  0011010100 -> ['H', '-', 'P', 'P', 'H']
[grover] Using iters = 13 (based on M=4)
[grover] Circuit qubits: 12

Top measured results:
0000100100   556  q0..q9=0010010000  ['H', '+', 'P', 'H', 'H']  VALID
0010101100   501  q0..q9=0011010100  ['H', '-', 'P', 'P', 'H']  VALID
0010100100   495  q0..q9=0010010100  ['H', '+', 'P', 'P', 'H']  VALID
0000101100   472  q0..q9=0011010000  ['H', '-', 'P', 'H', 'H']  VALID
1000101000     1  q0..q9=0001010001  ['H', 'P', 'P', 'H', 'P']  
0010011110     1  q0..q9=0111100100  ['P', '-', '+', 'P', 'H']  
0101110111     1  q0..q9=1110111010  ['-', '+', '-', '+', '+']  
0011011001     1  q0..q9=1001101100  ['+', 'P', '+', '-', 'H']  
0001110111     1  q0..q9=1110111000  ['-', '+', '-', '+', 'H']  
1110010110     1  q0..q9=0110100111  ['P